In [2]:
# packages
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Simple simulations

In [3]:
# define Black Scholes model as comparison/validation
def black_scholes(S0, K, T, r, sigma):
    d1 = (np.log(S0/K) + (r + 0.5 * sigma**2)*T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    return S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

In [8]:
# MC simulation for a vanilla call
def monte_carlo_european_call(S0, K, T, r, sigma, N):
    np.random.seed(42)

    Z = np.random.standard_normal(N)
    ST = S0 * np.exp((r - 0.5 * sigma**2)*T + sigma*np.sqrt(T)*Z)

    # payoff
    payoffs = np.maximum(ST - K, 0)

    # payoff mean
    price = np.exp(-r*T) * np.mean(payoffs)

    return price

Parameters : 
- $S_0$ : initial price (in USD)
- $K$ : strike (in USD)
- $T$ : time horizon (here, 1 year)
- $r$ : risk-neutral interest rate
- $sigma$ : volatility
- $N$ : number of simulations

In [15]:
# example with parameters
S0 = 100
K = 105
T = 1
r = 0.05
sigma = 0.2
N = 20000

In [11]:
mc_price = monte_carlo_european_call(S0, K, T, r, sigma, N)
black_scholes_price = black_scholes(S0, K, T, r, sigma)

In [12]:
print(f"Monte Carlo price : {mc_price:.2f}")
print(f"Black Scholes : {black_scholes_price:.2f}")

Monte Carlo price : 8.09
Black Scholes : 8.02


In [13]:
### ADD CONFIDENCE INTERVALS

In [ ]:
# visualization
#######

# Monte Carlo simulations with variance reduction methods

## Antithetic variates

In [ ]:
def monte_carlo_call_antitethic(S0, K, T, r, sigma, N):
    np.random.seed(42)

    Z = np.random.standard_normal(N // 2)

    # antitethic variables
    ST_1 = S0 * np.exp((r - 0.5 * sigma**2)*T + sigma*np.sqrt(T)*Z)
    ST_2 = S0 * np.exp((r - 0.5 * sigma**2)*T + sigma*np.sqrt(T)* (-Z))

    # associated payoff
    payoff_1 = np.maximum(0, ST_1 - K)
    payoff_2 = np.maximum(0, ST_2 - K)

    # average of antithetic variables
    averagepayoff = (payoff_1 + payoff_2) / 2

    # final measures
    price = np.exp(-r * T) * np.mean(averagepayoff)
    std_error = np.std(averagepayoff) / np.sqrt(N)

    # confidence interval 95%
    lowerbound = price - 1.96 * std_error * np.exp(-r * T)
    upperbound = price + 1.96 * std_error * np.exp(-r * T)

    return price, lowerbound, upperbound

In [23]:
# using the same parameters as previously
mc_price_antithetic, lower, upper = monte_carlo_call_antitethic(S0, K, T, r, sigma, N)
print(f"Monte Carlo (simple) : {mc_price:.2f}")
print(f"Monte Carlo (antithetic) : {mc_price_antithetic:.2f}")
print(f"Black-Scholes : {black_scholes_price:.2f}")

Monte Carlo (simple) : 8.09
Monte Carlo (antithetic) : 8.05
Black-Scholes : 8.02


In [25]:
# confidence intervals
print(f"Confidence interval at 95% : [{lower:.2f} ; {upper:.2f}]")

Confidence interval at 95% : [7.94 ; 8.15]


In [20]:
### visualization

## Control variates